In [11]:
import pandas as pd
import subprocess
import os, io, getpass
import psutil

In [8]:
def __MonitorNvidiaGPU():
    '''
        Function that monitors Running Processes on Nvidia GPU
        Returns a DataFrame (pid, process_name, cmdline, used_gpu_memory, utilization)
    '''
    getGPUProcesses = 'nvidia-smi pmon -c 1 -s mu'

    proc = subprocess.Popen(getGPUProcesses, shell=True, stdout=subprocess.PIPE)
    output = proc.stdout.read().decode('utf-8').split('\n')

    # Remove the line describing the units of each feature
    del output[1]
    # convert to csv format...
    output[0] = output[0].replace('# ', '')
    output = [line.strip() for line in output]
    output = [','.join(line.split()) for line in output]
    # ...and drop the command feature (will be added later)...
    output = [','.join(line.split(',')[:8]) for line in output]
    # ...and convert to DataFrame
    procsGPU = pd.read_csv(io.StringIO('\n'.join(output)), header=0)
    # procsGPUFeats = self.__GetProcessAttributes(procsGPU.pid.values)



    # return procsGPU.merge(procsGPUFeats, on='pid', how='inner')
    return procsGPU

In [9]:
def __MonitorCPU(mostImportant=10):
    procs = []

    for proc in psutil.process_iter():
        if proc.pid == os.getpid(): continue 
        if proc.status() != 'running': continue 
        try:
            if getpass.getuser() == proc.username():
                procs.append({'pid':            proc.pid,
                            'cpu_percent':    proc.cpu_percent(),
                            'memory_percent': proc.memory_percent(),
                            'name':           proc.name(),
                            'exe':            proc.exe(),
                            'status':         proc.status()
                            })
        except psutil.AccessDenied:
            continue

    process_log = pd.DataFrame(procs, columns=['pid', 'cpu_percent', 'memory_percent', 'name', 'exe', 'status'])

    if not process_log.empty:
        print ('Logging Data')
        return process_log.sort_values(['memory_percent'], ascending=False)[:mostImportant]
    else:
        print ('Nothing to Log')
        return process_log

In [ ]:
df1 = __MonitorNvidiaGPU()
df1

In [ ]:
df2 = __MonitorCPU()
df2

In [ ]:
df1.pid

In [ ]:
df2.pid

In [ ]:
df1.merge(df2, on='pid', how='inner')

In [ ]:
import datetime

In [ ]:
from time import gmtime, strftime, sleep
while (True):
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()), end='\r', flush=True)
    time.sleep(1)